In [1]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [2]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [3]:
train, dev = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
train.shape, dev.shape

((960, 8), (240, 8))

In [5]:
df['rating'].value_counts()

AAA     153
A-      150
A+      126
BBB+    115
A       110
BBB      90
AA       74
AA+      71
BBB-     65
BB+      57
AA-      54
BB       50
BB-      29
B+       20
B        12
B-       12
C        12
Name: rating, dtype: int64

In [6]:
categories = ['AAA', 'AA+', 'AA-', 'AA']

In [7]:
dev = dev.query('rating in @categories')
train = train.query('rating in @categories')

In [8]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [9]:
train.shape, dev.shape

((274, 8), (78, 8))

In [10]:
nlp = spacy.load('ru_core_news_lg')

In [11]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [12]:
create_docbin(dev).to_disk("dev.spacy")
create_docbin(train).to_disk("train.spacy")

In [13]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: subsym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       46.28    0.46
  0     200          35.15       80.46    0.80
  1     400          17.09       86.73    0.87
  2     600          11.32       91.04    0.91
  2     800           5.54       92.44    0.92
  3    1000           1.96       92.16    0.92
  4    1200           3.13       92.97    0.93
  5    1400           2.48       92.41    0.92
  5    1600           2.01       92.95    0.93
  6    1800           1.97       92.93    0.93
  7    2000           0.71       92.17    0.92
  8    2200           1.15       92.03    0.92
  8    2400           1.53       92.30    0